# Ambientando Fiestas
Este notebook tiene como objetivo importar toda la información necesaria para poder hacer un posterior análisis y ciencia de datos de la información obtenida de la organización de eventos Ambientando Fiestas

<a name='1'></a>
## 1 - Packages

In [57]:
from office365.runtime.auth.client_credential import ClientCredential
from office365.sharepoint.client_context import ClientContext
# Configuración de SharePoint
from msal import ConfidentialClientApplication
import pandas as pd 
import matplotlib.pyplot as plt

### Configurando el diccionario de credenciales

In [58]:
#%%writefile ~/.microsoft/credentials 
##credentials =  {}

<a name='3'></a>
## Configuraciones de sharepoint




In [59]:
sharepoint_config ={
"tenant" : "https://ambientandofiestas.sharepoint.com" ,
"site" :"Ambientandofiestas"
}   

### Guardando credenciales y configuraciones

In [60]:
import json
import os

path = "~/.microsoft" 

credential_path = {
    "path" :f"{path}/credentials",
    "data": credentials
}
config_path = {
    "path" : f"{path}/sharepoint_settings",
    "data" : sharepoint_config
}

paths = [credential_path,config_path]

for file_path in paths: 
    expanded_root = os.path.expanduser(file_path['path'])
    with (open(expanded_root,'w') as file):
        json.dump(file_path['data'],file)



### Getting our credentials and config

In [61]:
# Reading jsons data

#Credentials
expanded_root = os.path.expanduser(credential_path['path'])
with open(expanded_root, 'r') as file:
    credentials_data = json.load(file)
client_id = credentials_data["client_id"]
tenant_id = credentials_data["tenant_id"]
cert_thumbrs = credentials_data["cert_thumbrs"]
private_key = credentials['private_key']

#Sharepoint_config
expanded_root = os.path.expanduser(config_path['path'])
with open(expanded_root, 'r') as file:
    config_data = json.load(file)
tenant = config_data['tenant']
site = config_data['site']

'''
print (client_id)
print(tenant_id)
print(cert_thumbrs)
print(private_key)
'''


'\nprint (client_id)\nprint(tenant_id)\nprint(cert_thumbrs)\nprint(private_key)\n'

### Getting the token to makes the api calls

In [62]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend

cert = {
    "private_key" : private_key,
    "thumbprint": cert_thumbrs
}

authority = f"https://login.microsoftonline.com/{tenant_id}"
scope = [f"{tenant}/.default"]  # Adjust the scope to sharepoint

# Create app instance
app = ConfidentialClientApplication(
    client_id,
    authority=authority,
    client_credential=cert
)

# Get token
result = app.acquire_token_for_client(scopes=scope)

if 'access_token' in result:
    token = result['access_token']
    print("Access token:", token)
else:
    print("nosepudo")
    print("Getting token access failed. Error:", result.get("error"), result.get("error_description"))

Access token: eyJ0eXAiOiJKV1QiLCJub25jZSI6InV1VDhnU09iT25FUEdjX0tCWnBKN1VoRlBHY1VhMVhYWmlWSGw3NThkSmsiLCJhbGciOiJSUzI1NiIsIng1dCI6IkwxS2ZLRklfam5YYndXYzIyeFp4dzFzVUhIMCIsImtpZCI6IkwxS2ZLRklfam5YYndXYzIyeFp4dzFzVUhIMCJ9.eyJhdWQiOiJodHRwczovL2FtYmllbnRhbmRvZmllc3Rhcy5zaGFyZXBvaW50LmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0L2FkNDhlNjljLWFiNjgtNGNjOS04N2MxLTYxNTcxMWI4ZWIyNC8iLCJpYXQiOjE3MTgyMzE0NjAsIm5iZiI6MTcxODIzMTQ2MCwiZXhwIjoxNzE4MjM1MzYwLCJhaW8iOiJFMk5nWUxnWjRUT3AyRUdoWE5Qd1d1bmJqS25QQUE9PSIsImFwcF9kaXNwbGF5bmFtZSI6ImNvbmV4aW9uX3BhbmRhcyIsImFwcGlkIjoiZmU4ZDE5MWMtMzYzNy00M2FmLTk3NTItNzgzMzYyZDU1OWJkIiwiYXBwaWRhY3IiOiIyIiwiaWRwIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvYWQ0OGU2OWMtYWI2OC00Y2M5LTg3YzEtNjE1NzExYjhlYjI0LyIsImlkdHlwIjoiYXBwIiwib2lkIjoiZTFiZmFlYzgtMzgyZi00OTU3LThhY2YtOWM5MmZjNzI5MThhIiwicmgiOiIwLkFYd0FuT1pJcldpcnlVeUh3V0ZYRWJqckpBTUFBQUFBQVBFUHpnQUFBQUFBQUFDN0FBQS4iLCJyb2xlcyI6WyJTaXRlcy5SZWFkV3JpdGUuQWxsIl0sInNpZCI6ImQzODIwZTdiLTQ5YjAtNDY1Mi05NTI4LTMxOTU4YjE2MTI2NCIsInN1YiI6I

In [63]:
import requests

headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json;odata=verbose",
    "Content-Type": "application/json",
}
# Sharepoint URL site
site_url = f"{tenant}/sites/{site}"
response = requests.get(site_url,headers)
response

<Response [403]>

In [64]:
class SharePointList: 
    def __init__(self,url:str,list_name : str ,headers: dict[str,str]):
        self.site_url = url
        self.name = list_name
        self.headers = headers
        
    def get_all_items(self, retrieve_all_data:bool=True, fields:list[str]=[]) -> list[any]:    
        all_data = []
        api_url = self.site_url
        if fields:
            fields_string = ",".join(fields)
            api_url = f"{api_url}/_api/web/lists/getByTitle('{self.name}')/items?$select={fields_string}"
        else:
            api_url = f"{api_url}/_api/web/lists/getByTitle('{self.name}')/items"
        if (retrieve_all_data): 
            
            while api_url: 
                response = requests.get(api_url,headers =self.headers)
                if response.status_code ==200:
                    data = response.json()
                    all_data.extend(data['d']['results'])             
                    api_url = data['d']['__next'] if '__next' in data['d'] else None
                else:
                    print(f"Error fetching the data {response.status_code}")
                    break
            
        else:            
            response = requests.get(api_url,headers=self.headers)
            if response.status_code ==200:
                data = response.json()
                all_data.extend(data['d']['results'])     
            else:
                print(f"Error fetching the data {response.status_code}")
        return all_data

# Obteniendo la información

## Lists

### Pedidos

In [65]:
elements = ["Title","Estado","Usuario","Edad","Homenajeado","Tematica","Horario","FinEvento","Lugar","Created"]

pedidos_actuales = SharePointList(url=site_url,list_name="Pedidos",headers=headers)
pedidos_actuales_list = pedidos_actuales.get_all_items(retrieve_all_data=True,fields=elements)
df_pedidos_actuales = pd.DataFrame(pedidos_actuales_list)

pedidos_finalizados = SharePointList(url=site_url,list_name="Pedidos finalizados",headers=headers)
pedidos_finalizados_list = pedidos_finalizados.get_all_items(retrieve_all_data=True,fields=elements)
df_pedidos_finalizados = pd.DataFrame(pedidos_finalizados_list)

df_pedidos_sh= pd.concat([df_pedidos_actuales,df_pedidos_finalizados])

df_pedidos_sh=df_pedidos_sh.drop(["__metadata","Homenajeado"],axis=1)

df_pedidos_sh = df_pedidos_sh.rename(columns={"Horario":"Inicio_evento","FinEvento":"Fin_evento"})

df_pedidos_sh.head()

,Title,Estado,Usuario,Edad,Tematica,Inicio_evento,Fin_evento,Lugar,Created
0,7473,finalizado,Alejandrasarro@gmail.com,6,n/a,2024-05-19T14:23:00Z,2024-05-19T03:00:00Z,SUM,2023-09-08T14:27:24Z
1,8096,confirmado,pcouceiro88@gmail.com,3,No sabemos,2024-05-14T19:30:00Z,2024-05-14T22:00:00Z,Casa,2023-10-31T10:03:25Z
2,8222,confirmado,bolomariapaz@gmail.com,1,animalitos en safari,2024-06-30T16:00:00Z,2024-06-30T20:00:00Z,Casa,2023-11-08T15:09:40Z
3,8319,confirmado,daia2505@hotmail.com,1,mickey y sus amigos,2024-06-15T03:00:00Z,2024-06-15T03:00:00Z,Salón,2023-11-15T18:21:22Z
4,8584,confirmado,camilatovani52@gmail.com,1,Carrusel,2024-08-06T22:00:00Z,2024-08-07T01:00:00Z,Salón,2023-12-17T19:01:35Z


### Detalle Pedido

In [66]:
elements = ["id_detalle_pedido","id_pedido","Title","id_producto","Cantidad"]

detalle_pedido_actuales = SharePointList(url=site_url,list_name="Detalle pedido",headers=headers)
detalle_pedido_actuales_list = detalle_pedido_actuales.get_all_items(retrieve_all_data=True,fields=elements)
df_detalle_pedido_actuales = pd.DataFrame(detalle_pedido_actuales_list)

detalle_pedido_finalizado = SharePointList(url=site_url,list_name="Detalle pedido finalizado",headers=headers)
detalle_pedido_finalizado_list = detalle_pedido_finalizado.get_all_items(retrieve_all_data=True,fields=elements)
df_detalle_pedido_finalizado = pd.DataFrame(detalle_pedido_finalizado_list)

df_detalle_pedido= pd.concat([df_detalle_pedido_actuales,df_detalle_pedido_finalizado])
df_detalle_pedido=df_detalle_pedido.drop("__metadata",axis=1)
df_detalle_pedido.head()

Error fetching the data 503


,Title,id_producto,id_detalle_pedido,Cantidad,id_pedido
0,6986,5823,3259,1,850
1,6989,2972,3263,1,855
2,7000,2975,3273,1,870
3,7107,5823,3381,1,991
4,7143,2719,3422,1,1046


In [67]:
df_detalle_pedido.shape


(2529, 5)

### Colaboraciones por pedido

In [68]:
elements = ["id_colaborador","colaboracion","Title","estado_colaboracion","id_pedido"]

colaboraciones_por_pedido_actual = SharePointList(url=site_url,list_name="Colaboraciones por pedido",headers=headers)
colaboraciones_por_pedido_list = colaboraciones_por_pedido_actual.get_all_items(retrieve_all_data=True,fields=elements)
df_colaboraciones_por_pedido = pd.DataFrame(colaboraciones_por_pedido_list)


colaboraciones_por_pedido_finalizado = SharePointList(url=site_url,list_name="Colaboraciones por pedido finalizados",headers=headers)
colaboraciones_por_pedido_finalizado_list = colaboraciones_por_pedido_finalizado.get_all_items(retrieve_all_data=True,fields=elements)
df_colaboraciones_por_pedido_finalizado = pd.DataFrame(colaboraciones_por_pedido_finalizado_list)
df_colaboraciones_por_pedido = pd.DataFrame()
df_colaboraciones_por_pedido = pd.concat([df_colaboraciones_por_pedido,df_colaboraciones_por_pedido_finalizado])
df_colaboraciones_por_pedido=df_colaboraciones_por_pedido.drop(['__metadata','estado_colaboracion'],axis=1)
df_colaboraciones_por_pedido.head()




,Title,id_colaborador,colaboracion,id_pedido
0,7873,32.0,G,2268
1,7113,28.0,I,2303
2,6935,28.0,I,2335
3,6935,28.0,I,2336
4,7185,28.0,I,2338


## Obteniendo la información desde el e-commerce (woocommerce)

Realizamos la conexión con el e-commerce de la Ambientando Fiestas

In [71]:
from woocommerce import API

wcapi = API(
    url="https://ar.ambientandofiestas.com/",
    consumer_key="ck_a55e65b141c8d7dee13bcd3b5f88eae5f16fb2bd",
    consumer_secret="cs_b89e3fdb3b5061845f78c8d875d454c269ff8037",
    version="wc/v3",
    timeout=200 
)

Obtener pedidos desde woocommerce

In [72]:

df_pedidos_wc = pd.DataFrame()
registros_por_pagina = 100


pagina = 1
respuesta = True
# Aumenta el timeout a 30 segundos
while respuesta:
    # Hace la solicitud a la API

    respuesta = wcapi.get("orders", params={"per_page": registros_por_pagina, "page": pagina}).json()
    if not respuesta:
        break

    # Convierte la respuesta en DataFrame y la agrega al DataFrame existente
    df_temp = pd.DataFrame(respuesta)
    df_pedidos_wc = pd.concat([df_pedidos_wc, df_temp], ignore_index=True)

    # Incrementa el número de página
    pagina += 1

# Muestra los primeros registros para verificar



In [73]:
df_pedidos_wc.head(3)

,id,parent_id,status,currency,version,prices_include_tax,date_created,date_modified,discount_total,discount_tax,...,payment_url,is_editable,needs_payment,needs_processing,date_created_gmt,date_modified_gmt,date_completed_gmt,date_paid_gmt,currency_symbol,_links
0,10008,0,on-hold,ARS,8.0.1,False,2024-06-12T18:25:46,2024-06-12T18:25:46,0,0,...,https://ar.ambientandofiestas.com/checkout/ord...,True,False,True,2024-06-12T18:25:46,2024-06-12T18:25:46,None,None,$,{'self': [{'href': 'https://ar.ambientandofies...
1,10007,0,processing,ARS,8.0.1,False,2024-06-12T13:35:45,2024-06-12T13:37:11,0,0,...,https://ar.ambientandofiestas.com/checkout/ord...,False,False,True,2024-06-12T13:35:45,2024-06-12T13:37:11,None,2024-06-12T13:37:11,$,{'self': [{'href': 'https://ar.ambientandofies...
2,10005,0,processing,ARS,8.0.1,False,2024-06-12T11:06:33,2024-06-12T11:07:29,0,0,...,https://ar.ambientandofiestas.com/checkout/ord...,False,False,True,2024-06-12T11:06:33,2024-06-12T11:07:29,None,2024-06-12T11:07:29,$,{'self': [{'href': 'https://ar.ambientandofies...


La columna billing es una columna del tipo json.

In [74]:
df_pedidos_wc['billing'].head()

0    {'first_name': 'Andrea Soledad', 'last_name': ...
1    {'first_name': 'Julieta trillo', 'last_name': ...
2    {'first_name': 'Natalia', 'last_name': 'Fernan...
3    {'first_name': 'Catherine Curillo', 'last_name...
4    {'first_name': 'Agustina Alba', 'last_name': '...
Name: billing, dtype: object

Aplicamos una función para poder obtener todas las columnas dentro de la columnas billing

In [75]:
def getColumnsOfBilling(df_pedidos_woo_commerce):
    billing_list = []        
    
    for item,column in df_pedidos_woo_commerce.iterrows():
        
        title = f", 'id': '{column['id']}'}}"
        dict =str(column['billing'])[:-1] + title
        billing_list.append(dict)
    return billing_list

In [76]:
def convert_to_dict(item,index):
    try:
        return json.loads(item)     
    except:
        print(f"No se pudo con el item: {index}")
        return False

In [77]:
billing_list = getColumnsOfBilling(df_pedidos_wc)
id_list = df_pedidos_wc['id'].tolist()
clean_list = [item.replace('\\', '\\\\').replace("'", '"') for item in billing_list]

total_invalid_items = 0    
dict = []
for index, item in enumerate(clean_list):
    if(convert_to_dict(item,index)):
        dict.append(convert_to_dict(item,index))
    else:
        total_invalid_items += 1
print(f"Total items no válidos {total_invalid_items}")

df_billings_ = pd.DataFrame(dict)
df_billings_ = df_billings_.rename(columns={'billing_date':'Inicio_evento_wc'})
df_billings_['id'] = df_billings_['id'].astype('Int64')
df_billings_['Inicio_evento_wc'] = pd.to_datetime(df_billings_['Inicio_evento_wc'],utc=True)
#df_billings = df_billings_[['id','Inicio_evento_wc']]
df_billings_.head(3)
df_billings_.info()
df_billings = df_billings_.copy()

No se pudo con el item: 965
No se pudo con el item: 1186
No se pudo con el item: 1574
No se pudo con el item: 1738
No se pudo con el item: 2232
No se pudo con el item: 2449
No se pudo con el item: 3023
No se pudo con el item: 3109
No se pudo con el item: 3229
No se pudo con el item: 3271
No se pudo con el item: 3303
No se pudo con el item: 3312
Total items no válidos 12
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3595 entries, 0 to 3594
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   first_name          3595 non-null   object             
 1   last_name           3595 non-null   object             
 2   company             3595 non-null   object             
 3   address_1           3595 non-null   object             
 4   address_2           3595 non-null   object             
 5   city                3595 non-null   object             
 6   state               3595 no

In [78]:
df_merge_pedidos = pd.merge(df_pedidos_wc,df_billings,left_on='id',right_on='id',how='inner')
df_woo_commerce_complete = df_merge_pedidos.copy()
df_woo_commerce_complete = df_woo_commerce_complete.rename(columns={"id":"Title"})


In [79]:
df_pedidos_wc = df_woo_commerce_complete.copy()

Dejamos los datos que nos sirven

## Saving into csv to analyse the dataframes

In [80]:
df_pedidos_wc.to_csv('pedidos_wc.csv',index=False)
df_pedidos_sh.to_csv('pedidos_sh.csv',index=False)

In [ ]:
df_pedido_detalle_pedidos_tematica_wrang['Tematica'].unique()

array(['margaritas', 'sin definir', 'minnie', 'selva',
       'flores y colores pasteles', 'a definir', 'mickey safari',
       'mickey mouse safari', 'winnie the pooh y amigos bebés',
       'osito o safari a definir', 'n/a', 'hello kitty', 'jugueteria',
       'vikingos', 'caballerisa', 'conejos', 'paw patrol',
       'granja de zenon', 'vaca lola', 'monsters inc', 'safari',
       'no sabemos', 'unicornio', 'plim plim', 'winnie pooh',
       'star wars (galaxia, darth vader, jedi y stormtroppers)',
       'aun no se', 'bautismo', 'mujer maravilla', 'harry potter',
       'autos', 'aby casas de muñecas', 'ciervita- bosque',
       'animalitos en safari', 'temática ositos', 'baby yoda',
       'sirenita ariel', 'mickey y sus amigos', 'neon',
       'animales del bosque para nena',
       'a definir (pero creo q va a estar relacionado a encanto)',
       'la sirenita', 'tranformer y barbie', 'toy story', 'dinosaurios',
       'aviones', 'masha y el oso', 'frozen', 'la sirenita o la gra

With all the uniques values, we asked GPT to create categories to eliminate words we don't need. The result was this

In [ ]:
mapeo = {
    'Barbie': ['Barbi', 'Barbie', 'BARBIR'],
    'Disney': ['disney','disney playero'],
    'bambi': ['el bambi central,','bambi'],
    'tortugas ninja': [ 'tortugas ninja','tortugas ninjas'],
    
    'Mickey': ['Mickey', 'MIKEY', 'MICKEY',  'MICHEY', 'MICKEY MOUSE'],
    'bob esponja':['bob esponja','bob espona','bob esponka'],
    "jefe en pañales": ['jefe en pañales /','jefe en pañales'],
    "Minnie": ['Minni', 'MINNIE', 'MINIE','MINNI','MINNIE MOUSE','minnie','MINNI'],
    'Safari': ['Safari', 'SAFARI', 'ZAFARI', 'SADARI', 'ANIMALITOS SAFARI', 'SAFARI MICKEY', 'SAFARI PARA NIÑA', 'SAFARI DE MIKEY', 'SELVA SAFARI'],
    'Sirenita': ['SIRENA BAJO EL MAR', 'cola de sirena','Sirenita', 'LA SERENITA', 'SIRENITA ARIEL', 'LA SIRENITA', 'SIRENAS', 'SIRENITA ARIEL BEBÉ', 'LA SERENITA', 'SIRENA BAJO EL MAR'],
    'Frozen': ['Frozzens', 'Frozzen', 'FROZEN', 'FROZEN 2', 'FROOZEN', 'ELSA FROZEN 2!'],
    'Toy Story': ['Toy Story', 'TOY STORY', 'OY STORY', 'TOY STORY 1 O 4', 'TOY STORY PARA NENA', 'TOY STORY COLORES PASTELES', 'TOY STORY 2','buzz lightyear'],
    'Selección Argentina': ['SELECCION ARGENTINA', 'ARGENTINA CAMPEON', 'ARGENTINA', 'MESSI-ARGENTINA','argentina campeon'],
    'Messi': ['MESSI', 'MESSI FUTBOL ARGENTINA', 'MESSI RAYA Y EL ULTIMO DRAGON'],
    'sin definir': ['por definir','no está definido aún','no lo sé aún','todavía no está decidido','fiesta de fin de año','quisiera ayuda para decir','no definido','consultar','aun no definido','sin definir', 'n/a', 'no sabemos', 'aun no se', 'no aplica', 'aún no definido', 'aún no lo sabemos', 'aún no definida', 'todavía no elegí temática', 'consultar', 'ninguna', 'no especifica', 'aun no definida', 'a definir', 'a definir (pero creo q va a estar relacionado a encanto)', 'a definir por whatsapp', 'a definir-cumple 1 añito', 'aun no difinida'],
    'Fútbol': ['inter miami','inter de miami','boca jrs','futbol','FÚTBOL (REAL MADRID)', 'FÚTBOL ARGENTINO'],
    'Princesas Disney': ['PRINCESAS DISNEY', 'PRINCESA JAZMIN', 'JAZMIN ALADIN', 'LAS PRINCESAS DE DISNEY', 'PRINCESA', 'PRINCESAS', 'CENICIENTA', 'PRINCESAS Y UNICORNIOS', 'PRINCESA PEACH'],
    'Superheroes': [ 'vengadores','spidey and his amazing friends', 'speackerman', 'spider-man', 'superman', 'batman',  'spiderman','spaiderman','vengadores y heroinas','súper héroe','deadpool','dead pool','súper héroes','SUPERHÉROES', 'HOMBRE ARAÑA-SUPERMAN-BATMAN-HULK', 'SUPER MARIO ODYSSEY', 'HOMBRE ARAÑA', 'SPIDERMAN', 'BATMAN', 'AVENGERS', 'SUPER MARIO BROS', 'SUPER HEROES', 'SUPER MARIO BROS', 'BATMAN/MUJER MARAVILLA/SPIDERMAN/IRON MAN', 'SPIDER-MAM', 'SUPERHEROES'],
    'Animales': ['animales','SELVA', 'ANIMALES DEL BOSQUE', 'ANIMALITOS DEL BOSQUE', 'GRANJA', 'ZOO', 'JIRAFAS', 'CONEJOS', 'ANIMALES DE LA SELVA', 'ANIMALITOS', 'ANIMALES NORDICOS', 'ANIMALITOS NORDICOS', 'GRANJA DE ZENON', 'GRANJA ROSA', 'GRNAJA', 'ANIMALES BEBES DE LA SELVA', 'ANIMALITOS DE LA SELVA BEBÉ', 'GRANJA (LA GRANJA DE OLI, FOTO ENVIADA)', 'LA GRANJA DE ZENON', 'LA GRANJA ROSA'],
    'Winnie the Pooh': ['winny poo bebe y sus amigos','winnie de poh','winnie the pooh','WINNIE THE POOH Y AMIGOS BEBÉS', 'WINNIE POOH', 'WINNIE POOH BEBÉ', 'PIGLET BEBE (CERDITO DE WINNIE POOH VERSION BEBE)'],
    'Hello Kitty': ['HELLO KITTY', 'HELLO KITTY.'],
    'Star Wars': ['STAR WARS', 'STARWARS', 'BABY YODA', 'BABY YODA/ GALAXIA', 'BABY JODA'],
    'Cars': ['CARS', 'CARS - MONTAJE CIRCULAR', 'CARS (RAYO Y MATE)', 'RAYO MCQUEEN/ MATE', 'CARS - RAYO MCQUEEN'],
    'Unicornios': ['unicorn party','UNICORNIO', 'UNICORNIOS Y ARCOIRIS', 'UNICORNIO ( COLORES PASTELES) Y PUEDE AGREGARSE ARCOIRIS', 'UNICORNIOS- ARCOÍRIS PASTEL', 'UNICORNIO'],
    'Arcoíris': ['ARCOÍRIS', 'arcoíris', 'arco iris', 'arcoíris color pastel', 'arcoíris boho', 'boho rainbow pastel celestes','ARCO IRIS BOHO', 'ARCOIRIS BOHO', 'ARCOIRIS', 'ARCOIRIS COLOR PASTEL', 'ARCOÍRIS Y OSITOS', 'ARCOÍRIS COLORES PASTELES', 'ARCOÍRIS PASTELES', 'NUBE Y ARCOÍRIS', 'ARCOIRIS PASTEL, ROSA PASTEL Y BLANCO'],
    'Monsters Inc': ['MONSTERS INC', 'MOSTER INC'],
    'Hadas': ['HADAS', 'TINKERBELL - HADAS', 'JARDIN DE HADAS', 'LA TEMATICA CARITAS Y ARCOIRIS'],
    'Baby Shower': ['BABY SHOWER', 'baby showe', 'descubrimiento de género/ baby  shower','BABY SHOWER ANTONIA', 'BABY SHOWER', 'BABY SHOWER TEMÁTICA ARCOÍRIS (COLORES PASTELES)', 'BABY SHWER', 'BABY SHOWER/ REVELACION DE GENERO', 'ES UN BABY SHOWER, DE OSITO'],
    'spa party': ['spa party', 'spa de chicas'],
    'Bautismo': ['BAUTISMO', 'BAUTISMO PRIMER AÑO', 'BAUTISMO - ANIMALES', 'BAUTISMO ANIMALITOS DE LA SELVA', 'ANGELITOS (BAUTISMO)'],
    'Halloween': ['HALLOWEEN', 'FIESTA DE HALLOWEEN 🎃', 'HALLOWEN', 'CUTE HALLOWEEN/', 'HALLOWEEN CON HELLO KITTY'],
    'Dinosaurios': ['DINOSAURIOS', 'jurassic world', 'jurassica','DINOSAURIOS', 'DINOSAURIO', 'DINOSAURIOS BEBÉS', 'DINOSAURIOS BEBES', 'DINOSAURIO T REX', 'JURASSIC PARK', 'JURASSICC WORLD,', 'JURASIK PARK', 'JURASIC PARK','dinos'],
    'Colores': ['COLORES', 'COLORES LISOS', 'CELESTE, BLANCO Y BEIGE O MARRON CLARITO', 'COLORES A DEFINIR', 'COLOR ROSA', 'COLORES PASTEL', 'COLORES PASTEL + ANIMALES DEL MAR', 'PLATEADO , ROJO VINO Y BLANCO', 'ROSAS LISOS', 'MARIPOSAS Y COLORES PASTEL', 'GATITOS COLOR PASTEL', 'MINNIE ROSA CLARITO, SALMÓN, 1 AÑITO', 'BRILLOS PLATEADOS Y TODO COLOR LILA Y VIOLETA .', 'GLOBOS AMARILLOS, CELESTES Y ALGUNOS NEGROS ( FEDE VIGIOVANI)', 'GLOBOS LILAS VIOLETAS ROSAS'],
    'Mar': ['FONDO DEL MAR', 'BAJO DEL MAR', 'FONDE DE MAR',  'DEBAJO DEL MAR', 'LA SIRENITA DEBAJO DEL MAR', 'LA SIRENITA BAJÓ DEL MAR', 'BAUTISMO ANIMALITOS DE LA SELVA', 'ANIMALES DEL BOSQUE. BAMBI.', 'LA SIRENITA COLORES PASTELES', 'SIRENITA ARIEL', 'LA SIRENITA', 'SIRENITA ARIEL BEBÉ', 'SIRENAS', 'SIRENAS. COLORES VIOLETA CELESTE ROSA Y BLANCO', 'MAR', 'BURBUJAS MAR'],
    'Harry Potter': ['HARRY POTTER', 'HARRY POTER'],
    'Osos': ['OSO', 'OSITOS', 'OSITO AVIADOR', 'OSITO MARINERO', 'OSO ROSA', 'OSO PANDA BEBÉ NIÑA', 'OSITOS ROSADOS', 'TEMÁTICA OSITOS', 'OSITOS (EN TONO CELESTE, BEIGE Y MARRÓN)', 'ANIMALITOS NORDICOS, LA CIERVITA ES LA IMAGEN PRINCIPAL', 'ANIMALITOS NORDICOS'],
    'Aviones': ['AVIONES', 'AVIONES Y AUTOS', 'AVION VINTAGE', 'AVONES'],
    'perros': ['perros', 'perritos', 'perritos', 'perros salchicha', 'perrito negro y yorkie'],
    'gabby y la casa de muñecas': ['aby casas de muñecas','gabby y la casa de muñecas', 'gabby casita de muñecas', 'gabby y la casa de muñecas.' ,'casita de gabby','gaby casita de muñecas'],
    'liloystich': ['stich', 'stitch', 'stich disney','lilo y stich'],
    'las pistas de blue': ['las pistas de blue', 'las pistas de blue y tu'],
    'peppa': ['peppa', 'pepa paig', 'pepa pig', 'pepa.', 'pepa.', 'pepa paig'],
    'moana': ['moana', 'moana bebe'],
    'conejo': ['conejitos	','conejitos','conejo', 'conejo peter rabbit', 'conejo piter rabbir', 'conejo con globos','conejita', 'conejita bebé'],
    'cumpleaños': ['40','15b años','15 años','1 año','fiesta de 40','cumple', '45 años','cumple de 18','cumpleañero','60 años','cumpleanos de 1','primer añito', 'primer añito', 'cumpleaños de 1','cumpleaños', 'cumpleaños gemelas 1 año', 'cumple de 6 años', 'cumple 6', 'cumpleaños de 1', 'cumpleaños 40','cumpleaños de 40', 'cumple 40 años', 'cumple de 40', 'cumpleaños de 40 (hombre)', 'cumple 40', 'cumple 70 años violeta', 'cumple 2 años', 'cumpleaños de 2 años', 'cumpleaños de 4 años', 'cumpleaños de un año', 'cumpleaños de 1', 'cumpleaños de 1 año', 'cumpleaños de gemelas 1 año', 'cumpleaños de 18', 'cumple 18', 'cumpleaños de 15', 'cumple de 15 años.', 'cumpleaños de 15 años', 'cumpleaños de 1'],
    'paw patrol': ['paw patrol', 'paw atrol','pow patrl','skye de paw patrol', 'skye'],
    'hot wheels': ['hotwheesl','hotwheels','hoy wheels', 'hot wells', 'hot wheels'],
    'sing ven y canta': ['sing ven y canta!', 'sing ven y canta', 'sing 2 ven y canta!', 'sing 2'],
    'shimmer': ['shimmer', 'shimmer con cartel led', 'shimmer con cartel led boca'],
    'plim plim': ['plim plim', 'plim plim cumple 1'],
    'carrusel': ['carrusel', 'carrusel niña'],
    "encanto": ['encanto'],
    "merlina": ['wednesday - merlina y su hermano','merlina'],
    "espacial":['espacial'],
    "zorro":['zorrito','zorritos nórdico durmiendo', 'zorrito durmiendo en el cielo'],
    "cocomelon": ['cocomelon'],
    "bosque": ['nimalitos del bosque','bosque','bosque encantado']

    # Añadir más categorías y variantes según sea necesario
}




In [ ]:
def normalize_tematica(tematica, mapeo):
    for categoria, variantes in mapeo.items():
        if any(variante.lower() in tematica.lower() for variante in variantes):
            return categoria.upper()
    return tematica  # Devuelve el tema original si no se encontró ninguna variante

In [ ]:
df_pedido_detalle_pedidos_tematica_wrang['Categoria']= df_pedido_detalle_pedidos_tematica_wrang['Tematica'].dropna().apply(lambda x : normalize_tematica(x,mapeo))
df_pedido_detalle_pedidos_tematica_wrang

,Title,Usuario,Edad,Tematica,Horario,FinEvento,Lugar,id_producto,Cantidad,Categoria
0,6983,danidelutier@gmail.com,1,margaritas,2024-05-11T15:00:00Z,2024-05-11T22:00:00Z,Casa,5823.0,1.0,MAR
1,6983,danidelutier@gmail.com,1,margaritas,2024-05-11T15:00:00Z,2024-05-11T22:00:00Z,Casa,2975.0,1.0,MAR
2,7005,micaelataranco@hotmail.com,2,sin definir,2024-03-03T15:00:00Z,2024-03-03T20:00:00Z,Casa,2936.0,1.0,SIN DEFINIR
3,7020,nrgiacumbo@gmail.com,1,minnie,2024-03-09T15:30:00Z,2024-03-09T18:30:00Z,Salón,2968.0,1.0,MINNIE
4,7047,paula.wal85@gmail.com,1,selva,2024-03-09T23:00:00Z,2024-03-09T03:00:00Z,SUM,2968.0,1.0,ANIMALES
...,...,...,...,...,...,...,...,...,...,...
2386,9033,pilipefaur@gmail.com,5,barbie,2024-02-22T17:30:00Z,2024-02-22T21:00:00Z,Casa,5969.0,3.0,BARBIE
2387,9033,pilipefaur@gmail.com,5,barbie,2024-02-22T17:30:00Z,2024-02-22T21:00:00Z,Casa,5823.0,1.0,BARBIE
2388,9033,pilipefaur@gmail.com,5,barbie,2024-02-22T17:30:00Z,2024-02-22T21:00:00Z,Casa,5829.0,1.0,BARBIE
2389,8971,cindy.alvarez@outlook.es,2,mickey,2024-02-25T15:00:00Z,2024-02-25T19:00:00Z,SUM,7979.0,1.0,MICKEY


In [ ]:
df_animales_clenaed = df_pedido_detalle_pedidos_tematica_wrang.groupby('Categoria').count()
df_animales_clenaed[(df_animales_clenaed['Title']>4) & (df_animales_clenaed['Title']<7)]

,Title,Usuario,Edad,Tematica,Horario,FinEvento,Lugar,id_producto,Cantidad
Categoria,,,,,,,,,
BAMBI,6,6,6,6,6,6,6,6,6
JEFE EN PAÑALES,5,5,5,5,5,5,5,5,5
SING VEN Y CANTA,5,5,5,5,5,5,5,5,5
argentin,5,5,5,5,5,5,5,5,5
circo,6,6,6,6,6,6,6,6,6
circus,6,6,6,6,6,6,6,6,6
comunión,5,5,5,5,5,5,5,5,5
el principito,5,5,5,5,5,5,5,4,4
sky,5,5,5,5,5,5,5,5,5


In [ ]:

df_pedido_detalle_pedidos_tematica_wrang.Categoria.unique()

array(['eige con detalles rosas claros, estilo boho.', 'SIN DEFINIR',
       'MINNIE', 'ANIMALES', 'COLORES', 'MICKEY', 'WINNIE THE POOH',
       'SAFARI', 'HELLO KITTY', 'jugueteria', 'vikingos', 'caballerisa',
       'MAR', 'PAW PATROL', 'vaca lola', 'MONSTERS INC', 'UNICORNIOS',
       'PLIM PLIM', 'STAR WARS', 'BAUTISMO', 'HARRY POTTER', 'autos',
       'GABBY Y LA CASA DE MUÑECAS', 'BOSQUE', 'OSOS', 'SIRENITA', 'neon',
       'BARBIE', 'TOY STORY', 'DINOSAURIOS', 'AVIONES', 'FROZEN',
       'CUMPLEAÑOS', 'dave and eva', 'super cute baby', 'CARRUSEL',
       'vampirina', 'CONEJO', 'DISNEY', 'LILOYSTICH', 'SUPERHEROES',
       'PRINCESAS DISNEY', 'animal print', 'sky', 'jazmin de aladin',
       'CARS', 'COCOMELON', 'wish', 'casamiento', 'zenon',
       'quiero mandarles la foto por wsp', 'ARCOÍRIS', 'principito',
       'FÚTBOL', 'música', 'PERROS', 'minions', 'SPA PARTY', 'rapunzel',
       'musical', 'pocahontas', 'MESSI', 'JEFE EN PAÑALES', 'PEPPA',
       'HADAS', 'BABY SHOWER'

In [ ]:
df_pedido_detalle_pedidos_tematica_wrang.Categoria.unique().size

300